In [1]:
from jw_package import *

import configuration completed !
Data configuration has been started !
Data configuration completed !
function configuration completed ! 
Good to go !


In [2]:
cdsm_sales = pd.read_csv('../data/05. data_sample_V4_WY', index_col = 0)
cdsm_sales[cdsm_sales['units'] > 3000]
cdsm_sales.drop(42763, inplace = True)
cdsm_sales.drop(84317, inplace = True)
cdsm_sales = cdsm_sales.reset_index(drop = True)

In [3]:
count_item_nbr = cdsm_sales.groupby('item_nbr').size().reset_index(name = 'count')
count_item_nbr = count_item_nbr.sort_values( by = ['count'], ascending = False).reset_index(drop = True)

In [4]:
pivot_data = cdsm_sales.pivot_table(index=['store_nbr', 'item_nbr'],columns=['Cdsm_Ch'],values='units', aggfunc='sum')
pivot_data['d'] = ((pivot_data['Abnormal'] - pivot_data['Moderate']) / pivot_data['Moderate'] * 100).round(2)
pivot_data.drop(columns=['Abnormal', 'Moderate'], inplace=True)
pivot_diff = pivot_data

# get_correlation과 추이가 같은지 알아보기 위해 store_nbr별로 정리한 pivot_diff

In [5]:
pivot_diff.head(10)

Cdsm_Ch                  d
store_nbr item_nbr        
1         9          -2.18
          28         -0.53
          40        -11.85
          47         11.87
          51         86.38
          89        261.76
          93         30.75
          99         33.18
2         5         -43.96
          11        -43.68

#  그 중 code_sum에 민감하게 반응하는 item_nbr

In [7]:
pivot_diff[pivot_diff['d'].apply(lambda x: abs(x)) >= 50].head(5)

Cdsm_Ch                  d
store_nbr item_nbr        
1         51         86.38
          89        261.76
2         93         58.10
3         93         98.84
          102       187.50

# 2012년의 1번 store에서 get_correlation을 test해보자

In [8]:
store_1 = get_correlation(1, 2012)

C:\Users\WJ\Anaconda3\lib\site-packages\scipy\stats\stats.py:2998: RuntimeWarning: Mean of empty slice.
  mx = x.mean()
C:\Users\WJ\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\WJ\Anaconda3\lib\site-packages\scipy\stats\stats.py:2999: RuntimeWarning: Mean of empty slice.
  my = y.mean()
C:\Users\WJ\Anaconda3\lib\site-packages\scipy\stats\stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\Users\WJ\Anaconda3\lib\site-packages\scipy\stats\stats.py:5240: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


In [9]:
show_me_pearson(store_1)

(1, 2012, -1, -1)
	 (9, 'avgspeed', 364) (0.029413892409414884, 0.5759077562060245)
	 (9, 'cool', 362) (-0.06507347742720462, 0.21678034409391772)
	 (9, 'depart', 0) (nan, nan)
	 (9, 'dewpoint', 363) (-0.16263173725428326, 0.0018798601109756173)
	 (9, 'heat', 362) (0.1962103722989204, 0.00017208151618507012)
	 (9, 'preciptotal', 303) (-0.07761708837800624, 0.17781222909816158)
	 (9, 'resultdir', 363) (0.05088556611367485, 0.33365257830459105)
	 (9, 'resultspeed', 363) (0.048113484264033224, 0.36068909407025174)
	 (9, 'sealevel', 361) (0.10930797729412822, 0.03790572490368731)
	 (9, 'snowfall', 0) (nan, nan)
	 (9, 'stnpressure', 363) (0.10580652394994502, 0.04394758191932755)
	 (9, 'sunrise', 0) (nan, nan)
	 (9, 'sunset', 0) (nan, nan)
	 (9, 'tavg', 362) (-0.1777482034736896, 0.0006805793222412667)
	 (9, 'tmin', 364) (-0.1657085413387121, 0.00151077966541003)
	 (9, 'wetbulb', 360) (-0.16325684039123894, 0.0018855999939363544)
	 (28, 'avgspeed', 364) (0.06836867776596853, 0.1931119899051

### 맨 위 튜플은 (가게번호, 연도, 시작달, 끝달)을 의미한다
### 들여쓰기 된 튜플들은 (item_nbr, weather_feature, missing data를 거른 샘플 수)를 의미한다
### 그 옆의 수치들은 (0~1 사이의 피어슨 상관계수, 0~1 사이의 p-value)를 뜻한다
### 왼쪽 수치는 높으면 높을수록, 오른쪽 수치는 낮으면 낮을수록 상관관계가 강한 것을 의미한다
### (jw/good_correlation.txt 참조)

# 89번 item에 주목하면,
# tmin, wetbulb, dewpoint와 상관관계가 강한 것을 확인할 수 있다

---------------

# 결론
> 1. ### codesum으로 Differnece(%)를 관찰 후 어떤 feature가 영향을 끼치는 지는 get_correlation으로

> 2. 우리가 해야 할 Linear regression modeling은 아래와 같지 않을까 생각한다
$$ A*w = b$$
    * A = M X N matrix that row = tuples, column = features
    * w = N vector that weight for each feature
    * b = M vector that label for desired 'units'
    
    
    위의 process가 w의 가중치를 구하는데 도움이 되지 않을까 (만약 중간에 논리적인 오류가 없다면)